In [1]:
import numpy as np
import pandas as pd
import json
import re

from transformers import AutoTokenizer, AutoModel
from sklearn.preprocessing import OneHotEncoder
from konlpy.tag import Okt
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df_nlp = pd.read_csv("C:/Users/pc/Model/Model-7/dataset/추천시스템_데이터.csv")

# JSON 문자열을 배열로 변환
df_nlp['설명_벡터'] = df_nlp['설명_벡터'].apply(lambda x: np.array(json.loads(x)))
df_nlp['색상_벡터'] = df_nlp['색상_벡터'].apply(lambda x: np.array(json.loads(x)))

# 데이터프레임의 열에 대해 apply 함수를 사용하여 문자열 벡터를 배열로 변환
df_nlp['원핫인코딩'] = df_nlp['원핫인코딩'].apply(lambda x: np.array([float(num) for num in x.strip('[]').split()]).reshape(1, -1))

df_nlp.head(5)

,꽃,월,계절,꽃말,설명,설명_벡터,색상_벡터,원핫인코딩,색상
0,각시붓꽃,3,봄,"부끄러움, 세련됨",부끄러움 세련됨. 각시라 하면 이제 막 시집 온 새색시를 연상케 한다. 그래서인지 ...,"[[0.07368195801973343, 0.15451766550540924, 0....","[[0.15296272933483124, 0.3076050877571106, -0....","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",보라
1,감국,10,가을,그윽한 향기,그윽한 향기. 가을 산야는 국화과 식물들 차지다. 특히 노란 꽃으로 향기까지 일품인...,"[[0.08965592086315155, 0.2825712561607361, 0.1...","[[0.11612437665462494, 0.3139854669570923, -0....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",노랑
2,개나리,3,봄,"희망,깊은 정, 달성",희망깊은 정 달성. 우리나라 전역에서 봄 소식을 가장 먼저 알려주는 대표적인 꽃 개...,"[[0.13718879222869873, 0.23945458233356476, 0....","[[0.11612437665462494, 0.3139854669570923, -0....","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",노랑
3,개나리,4,봄,희망,희망. 개나리 봄을 알리는 전령사 하면 가장 먼저 떠오르는 꽃이다. 나리나리 개나리...,"[[-0.06287881731987, 0.3564964234828949, 0.256...","[[0.11612437665462494, 0.3139854669570923, -0....","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",노랑
4,갯개미취,9,가을,추억,추억. 옛날 일 따위는 깨끗이 잊는 사람들이 많은 가운데서도 당신은 옛 일을 어제 ...,"[[0.17346309125423431, 0.0834614634513855, 0.0...","[[0.15296272933483124, 0.3076050877571106, -0....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...",보라


In [4]:
# Klue-RoBERTa 모델과 토크나이저 불러오기
model_name = 'klue/roberta-small'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 월/계절 원핫인코딩
encoder = OneHotEncoder()
encoded_data = encoder.fit_transform(df_nlp[['월', '계절']]).toarray()  # '월'과 '계절' 컬럼을 원핫인코딩
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['월', '계절'])) # 원핫인코딩된 결과를 새로운 데이터프레임으로 변환

okt = Okt()

# 확장된 색상 사전
color_synonyms = {
    '갈색': ['갈색', '브라운', '갈색의', '브라운색', '갈색이', '갈색이다', '갈색을', '갈색으로'],
    '노랑': ['노랑', '노란', '노란색', '황색', '노랑색', '노랗다', '노랗게', '노란빛', '노란 빛','누런'],
    '보라': ['보라', '보라색', '자주색', '보라빛', '보랏빛', '보라빛의', '보라빛이', '보라빛으로'],
    '분홍': ['분홍', '핑크', '분홍색', '핑크색', '분홍빛', '분홍빛의', '분홍빛이', '분홍빛으로'],
    '빨강': ['빨강', '빨간', '빨강색', '빨간색', '붉은', '붉은색', '붉은 빛', '붉다', '붉게'],
    '주황': ['주황', '주황색', '오렌지', '오렌지색', '주황빛', '주황빛의', '주황빛이', '주황빛으로'],
    '초록': ['초록', '초록색', '녹색', '초록빛', '초록의', '초록이', '초록으로'],
    '파랑': ['파랑', '파란', '파란색', '파랑색', '청색', '파랑빛', '파란빛', '파랗다', '파랗게','푸른','푸른빛', '푸른 색'],
    '흰색': ['흰색', '하양', '하얀', '백색', '하얀색', '하양색', '하얗다', '하얗게', '백색의', '백색이', '백색으로','흰']
}

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# 특수문자 및 띄어쓰기 제거 함수
def remove_special_characters(text):
    if not isinstance(text, str):
        text = str(text)

    # 특수문자 제거
    pattern = r'[^\w\s\.]' # 문자, 공백문자, 마침표 제외 제거
    clean_text = re.sub(pattern, '', text)

    # 한자 제거
    pattern = r'[\u4e00-\u9fff]' # 중국어 한자의 유니코드 시작과 끝 제거
    clean_text = re.sub(pattern, '', clean_text)
    clean_text = ' '.join(clean_text.split())
    return clean_text

In [5]:
# 문장을 벡터화 변환하는 함수
def get_sentence_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True) # 최대 길이 초과 시 잘라내기, 작은 경우 패딩 진행
    outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].detach().numpy() # 각 토큰 벡터의 첫 번째 벡터 확인 (CLS 토큰 벡터)

In [6]:
# 결합 벡터 생성 (설명 벡터, 색상 벡터, 원핫인코딩 벡터 결합)
def create_combined_vector(row):
    설명_벡터 = row['설명_벡터']
    색상_벡터 = row['색상_벡터']
    원핫인코딩 = row['원핫인코딩']

    combined_array = np.concatenate((설명_벡터, 색상_벡터, 원핫인코딩), axis=1)

    return combined_array

# 색상이 없는 경우 결합 벡터 생성 함수
def create_combined_vector_without_color(row):
    설명_벡터 = row['설명_벡터']
    원핫인코딩 = row['원핫인코딩']
    combined_array = np.concatenate((설명_벡터, 원핫인코딩), axis=1)
    return combined_array

In [7]:
# 사용자 입력 텍스트 분석 함수(월, 계절)
def extract_month_season(text):
    months = {
        '1월': 1, '2월': 2, '3월': 3, '4월': 4, '5월': 5, '6월': 6,
        '7월': 7, '8월': 8, '9월': 9, '10월': 10, '11월': 11, '12월': 12
    }
    seasons = {'봄': '봄', '여름': '여름', '가을': '가을', '겨울': '겨울'}

    month = None
    season = None

    for key, value in months.items():
        if key in text:
            month = value
            break

    for key in seasons.keys():
        if key in text:
            season = key
            break

    return month, season

In [8]:
# 입력 텍스트에서 색상을 추출하는 함수
def extract_color(text):
    text = text.lower()
    for color, synonyms in color_synonyms.items():
        for synonym in synonyms:
            if synonym in text:
                return color
    return None

In [9]:
# 형태소 분석기 초기화
okt = Okt()

# 입력 텍스트의 형태소 분석 및 명사형 확인 함수
def is_noun_phrase(text):
    # 형태소 분석
    pos_tags = okt.pos(text)
    # 디버깅을 위해 형태소 분석 결과 출력
    print(f"Text: {text}, POS Tags: {pos_tags}")
    # 모든 단어가 명사인지 확인
    for word, tag in pos_tags:
        if tag != 'Noun':
            return False
    return True

# 명사형 텍스트에 문맥 추가 함수
def add_context_if_noun(user_input):
    if is_noun_phrase(user_input):
        return user_input + "에 어울리는 꽃을 추천해." # " 꽃을 추천해"
    return user_input

In [10]:
# 사용자 입력을 최종 벡터화
def get_user_input_vector(user_input, user_month):
    input_text = remove_special_characters(user_input)
    input_embeddings = get_sentence_embedding(input_text)
    input_color_text = extract_color(user_input)
    month, season = extract_month_season(user_input)

    # 원핫 벡터 생성
    user_onehot_vector = np.zeros(len(encoder.get_feature_names_out(['월', '계절'])))
    if month is None and season is None: #사용자가 쓰는 날의 달 추출
        month = user_month
        if month in [3, 4, 5]:
            season = '봄'
        elif month in [6, 7, 8]:
            season = '여름'
        elif month in [9, 10, 11]:
            season = '가을'
        else:
            season = '겨울'
        month_idx = encoder.get_feature_names_out(['월', '계절']).tolist().index(f'월_{month}')
        season_idx = encoder.get_feature_names_out(['월', '계절']).tolist().index(f'계절_{season}')
        user_onehot_vector[month_idx] = 1
        user_onehot_vector[season_idx] = 1

    if season is None and month is not None: #사용자가 입력한 월 기준으로 계절 추출
        if month in [3, 4, 5]:
            season = '봄'
        elif month in [6, 7, 8]:
            season = '여름'
        elif month in [9, 10, 11]:
            season = '가을'
        else:
            season = '겨울'
        season_idx = encoder.get_feature_names_out(['월', '계절']).tolist().index(f'계절_{season}')
        user_onehot_vector[season_idx] = 1
        
    if month is not None: #텍스트에 입력된 월
        month_idx = encoder.get_feature_names_out(['월', '계절']).tolist().index(f'월_{month}')
        user_onehot_vector[month_idx] = 1
    if season is not None: #텍스트에 입력된 계절
        season_idx = encoder.get_feature_names_out(['월', '계절']).tolist().index(f'계절_{season}')
        user_onehot_vector[season_idx] = 1
    user_onehot_vector = user_onehot_vector.reshape(1, -1) #(1,16)

    # 벡터 결합
    if input_color_text:
        color_embeddings = get_sentence_embedding(input_color_text)
        user_vector = np.concatenate((input_embeddings, color_embeddings, user_onehot_vector), axis=1)
    else:
        user_vector = np.concatenate((input_embeddings, user_onehot_vector), axis=1)
    return user_vector

In [11]:
# 특정 이벤트에 가중치를 부여하는 함수 (각 행마다 개별 적용)
def apply_event_weight_for_row(user_input, row):
    event_weights = {
        '발렌타인': 1.2,
        '화이트': 1.2,
        '어버이날': 1.2,
        '어버이': 1.2,
        '부모': 1.2,
        '부모님': 1.2,
        '성년의날': 1.2,
        '성년': 1.2,
        '로즈데이': 1.2,
        '로즈': 1.2,
        '스승의날': 1.2,
        '선생': 1.2,
        '스승': 1.2,
        '선생님': 1.2,
        '연인': 1.2,
        '여자친구': 1.2,
        '남자친구': 1.2,
        '여친': 1.2,
        '남친': 1.2,
        '아내': 1.2,
        '와이프': 1.2,
        '남편': 1.2,
        '생일': 1.2,
        '기념일': 1.2,
        '사랑': 1.2
    }
    weight = 1.0

    for event, event_weight in event_weights.items():
        if event in user_input:
            weight *= event_weight
            break

    #사용자 월 추출
    month, season = extract_month_season(user_input)

    # 꽃의 설명 및 월,계절에 따른 추가 가중치 적용
    if any(event in row['설명'] for event in event_weights.keys() if event in user_input):
        weight *= 1.2
    # 월에 따른 추가 가중치 적용
    if month and isinstance(row['월'], int) and month == row['월']:
        weight *= 1.2
    # 계절에 따른 추가 가중치 적용
    if season and isinstance(row['계절'], str) and season == row['계절']:
        weight *= 1.2

    return weight

In [12]:
# 추천 시스템 함수 (코사인 유사도 기반)
def recommend_flower(user_input, user_month=None):
    user_input = add_context_if_noun(user_input)
    user_vector = get_user_input_vector(user_input, user_month)
    input_color = extract_color(user_input)
    df = df_nlp

    if input_color:
        # 색상이 명시된 경우 해당 색상의 꽃들로 필터링
        filtered_df = df[df['색상'] == input_color]
        filtered_df['최종_벡터'] = filtered_df.apply(lambda row: create_combined_vector(row), axis=1)
    else:
        # 색상이 명시되지 않은 경우 결합 벡터에서 색상 벡터를 제외
        filtered_df = df.copy()
        filtered_df['최종_벡터'] = filtered_df.apply(lambda row: create_combined_vector_without_color(row), axis=1)

    # 각 행에 대해 가중치 계산 및 유사도 산출
    filtered_df['유사도'] = filtered_df.apply(lambda row: cosine_similarity(user_vector, np.array(row['최종_벡터']).reshape(1, -1))[0][0] * apply_event_weight_for_row(user_input, row), axis=1)
    filtered_df['유사도'] = filtered_df['유사도'].astype(float)  # 숫자형으로 변환

    # 유사도를 기준으로 상위 3개의 꽃을 선택하고 중복된 꽃을 제거
    top3 = filtered_df.nlargest(3, '유사도').drop_duplicates(subset='꽃')

    # 만약 중복 제거 후 3개의 꽃이 되지 않는 경우, 다시 nlargest로 채우기
    if top3.shape[0] < 3:
        additional_top = filtered_df.nlargest(20, '유사도')  # 상위 20개 정도를 선택
        additional_top = additional_top[~additional_top['꽃'].isin(top3['꽃'])]
        top3 = pd.concat([top3, additional_top]).nlargest(3, '유사도').drop_duplicates(subset='꽃')

    return top3[['꽃', '꽃말', '유사도']].to_dict('records')

In [14]:
user_input= "어버이날에 붉은 꽃을 선물할래"
user_month = 5
recommendation = recommend_flower(user_input, user_month) 
recommendation

Text: 어버이날에 붉은 꽃을 선물할래, POS Tags: [('어버이날', 'Noun'), ('에', 'Josa'), ('붉은', 'Adjective'), ('꽃', 'Noun'), ('을', 'Josa'), ('선물', 'Noun'), ('할래', 'Verb')]


C:\Users\pc\AppData\Local\Temp\ipykernel_18376\570443495.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['최종_벡터'] = filtered_df.apply(lambda row: create_combined_vector(row), axis=1)
C:\Users\pc\AppData\Local\Temp\ipykernel_18376\570443495.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['유사도'] = filtered_df.apply(lambda row: cosine_similarity(user_vector, np.array(row['최종_벡터']).reshape(1, -1))[0][0] * apply_event_weight_for_row(user_input, row), axis=1)
C:\Users\pc\AppData\

[{'꽃': '카네이션', '꽃말': '여성의 애정, 모정', '유사도': 1.3793159289240342},
 {'꽃': '드라큘라 카네이션', '꽃말': '사랑과 존경', '유사도': 1.3714671777553376},
 {'꽃': '로마니 카네이션', '꽃말': '사랑과 존경', '유사도': 1.3714671777553376}]

In [21]:
def get_recommendations(self, user_input, user_month=None):
    recommendations = self.recommend_flower(user_input, df_nlp, user_month)
    return recommendations[['꽃', '꽃말', '유사도']].to_dict('records')

In [23]:
user_input= "어버이날에 붉은 꽃을 선물할래"
user_month = 5
recommendation = get_recommendations(user_input, user_month) 
recommendation

AttributeError: 'str' object has no attribute 'recommend_flower'